In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
df = pd.read_csv('slash.txt',sep='\t')


In [2]:
G = nx.DiGraph()
for index,rows in df.iterrows():
    u = rows['FromNodeId']
    v = rows['ToNodeId']
    s = rows['Sign']
    G.add_edge(u,v,sign=s)

In [3]:
in_pos = {}
in_neg = {}
in_pos_count ={}
in_neg_count = {}



for nodes in G.nodes():
    l = list(G.in_edges(nodes,data=True))
    pos=[]
    neg=[]
    
    for items in l:
        if items[2]['sign'] == 1:
            pos.append(items[0])
            
        else:
            neg.append(items[0])
    in_pos[nodes] = pos
    in_neg[nodes] = neg
    in_pos_count[nodes] = len(pos)
    in_neg_count[nodes] = len(neg)



out_pos = {}
out_neg = {}
out_pos_count= {}
out_neg_count = {}
for nodes in G.nodes():
    pos = []
    neg = []
    l = dict(G[nodes])
    for keys in l:
        if(l[keys]['sign'] == 1):
            pos.append(keys)
        else:
            neg.append(keys)
    out_pos[nodes] = pos
    out_pos_count[nodes] = len(pos)
    out_neg[nodes] = neg
    out_neg_count[nodes] = len(neg)

nx.set_node_attributes(G,in_pos,'in_pos')
nx.set_node_attributes(G,in_neg,'in_neg')
nx.set_node_attributes(G,out_pos,'out_pos')
nx.set_node_attributes(G,out_neg,'out_neg')
nx.set_node_attributes(G,out_pos_count,'out_pos_count')
nx.set_node_attributes(G,out_neg_count,'out_neg_count')
nx.set_node_attributes(G,in_pos_count,'in_pos_count')
nx.set_node_attributes(G,in_neg_count,'in_neg_count')


In [4]:
avg_in_pos = 0;
length = 0;
for item in in_pos_count:
    avg_in_pos += in_pos_count[item]
    length+=1
avg_in_pos = avg_in_pos/length
avg_in_pos

5.174969564158753

In [5]:
avg_in_neg = 0;
length = 0;
for item in in_neg_count:
    avg_in_neg += in_neg_count[item]
    length+=1
    

avg_in_neg = avg_in_neg/length
avg_in_neg

1.511200389578768

In [6]:
Nodes = G.nodes(data=True)

In [7]:
optimism_lower_than_avg = {}
for node in Nodes:
    oli=[]
    for item in node[1]['out_pos']:
        if in_neg_count[item]>avg_in_neg:
            oli.append(item)
    optimism_lower_than_avg[node[0]] = oli

In [8]:
nx.set_node_attributes(G, optimism_lower_than_avg , 'ol_array')

In [9]:
OH = {}
for node in Nodes:
    oli=[]
    for item in node[1]['ol_array']:
        if in_pos_count[item]<avg_in_pos:
            oli.append(item)
    OH[node[0]] = oli
nx.set_node_attributes(G, OH , 'oh_array')

In [10]:
optimism = {} 
for node in OH:
    X = len(OH[node])
    Y = len(optimism_lower_than_avg[node])
    if Y >0:
        optimism[node] = X/Y
    else:
        optimism[node] = 0

In [11]:
pessimism_h = {}
for node in Nodes:
    phi=[]
    for item in node[1]['out_neg']:
        if in_pos_count[item]>avg_in_pos:
            phi.append(item)
    pessimism_h[node[0]] = phi
nx.set_node_attributes(G, pessimism_h , 'ph_array')

In [12]:
pl = {}
for node in Nodes:
    oli=[]
    for item in node[1]['ph_array']:
        if in_neg_count[item]>avg_in_neg:
            oli.append(item)
    pl[node[0]] = oli

pl

{0: [25, 27, 28, 29],
 1: [530,
  675,
  678,
  679,
  680,
  681,
  682,
  683,
  685,
  686,
  689,
  690,
  692,
  695,
  697],
 2: [],
 3: [],
 4: [],
 5: [759, 760, 761, 762],
 6: [],
 7: [542, 794, 795, 796, 797],
 8: [69,
  70,
  85,
  87,
  88,
  125,
  231,
  622,
  662,
  668,
  869,
  871,
  872,
  873,
  874,
  875,
  876,
  877,
  881,
  884,
  885,
  886,
  887,
  888,
  895,
  899,
  900,
  902,
  905,
  906,
  907,
  908,
  909,
  911,
  912,
  914,
  916,
  919,
  921,
  922,
  923,
  924,
  925,
  926,
  927,
  928,
  929,
  930,
  932,
  933,
  937,
  938,
  940,
  941],
 9: [959],
 10: [],
 11: [],
 12: [],
 13: [],
 14: [29, 806, 1072, 1305, 1306, 1307, 1309, 1310, 1311, 1313],
 15: [],
 16: [906, 1253, 1268, 1433, 1434, 1438, 1439, 1441],
 17: [],
 18: [],
 19: [1461],
 20: [],
 21: [50,
  758,
  1469,
  1470,
  1471,
  1473,
  1475,
  1478,
  1479,
  1480,
  1483,
  1484,
  1485,
  1487,
  1490,
  1493,
  1494,
  1495,
  1498,
  1500],
 22: [],
 23: [],
 24: [],


In [13]:
pessimism = {} 
for node in pl:
    
    X = len(pl[node])
    Y = len(pessimism_h[node])
    if Y >0:
        pessimism[node] = X/Y
    else:
        pessimism[node] = 0

In [14]:
 nx.set_node_attributes(G, pessimism, 'pessimism')
 nx.set_node_attributes(G, optimism, 'optimism')

In [15]:
Edges = G.edges()


In [16]:
pesimism_u = {}
pesimism_v = {}
optimism_u = {}
optimism_v = {}
positivity_u = {}
positivity_v = {}
negativity_u = {}
negativity_v = {}

for edge in Edges:
    u = edge[0]
    v = edge[1]
    p1 = Nodes[u]['pessimism']
    p2 = Nodes[v]['pessimism']
    o1 = Nodes[v]['optimism']
    o2 = Nodes[v]['optimism']
    pos1 = len(Nodes[u]['ol_array'])
    pos2 = len(Nodes[v]['ol_array'])
    neg1 = len(Nodes[u]['ph_array'])
    neg2 = len(Nodes[v]['ph_array'])
    pesimism_u[edge] = p1
    pesimism_v[edge] = p2
    optimism_u[edge] = o1
    optimism_v[edge] = o2
    positivity_u[edge]= pos1
    positivity_v[edge] = pos2
    negativity_u[edge] = neg1
    negativity_v[edge] = neg2
    

In [17]:
nx.set_edge_attributes(G,  positivity_u, 'positivity_u')
nx.set_edge_attributes(G,  positivity_v, 'positivity_v')
nx.set_edge_attributes(G,  negativity_u, 'negativity_u')
nx.set_edge_attributes(G,  negativity_v, 'negativity_v')
nx.set_edge_attributes(G,  pesimism_u, 'pesimism_u')
nx.set_edge_attributes(G,  pesimism_v, 'pesimism_v')
nx.set_edge_attributes(G,  optimism_u, 'optimism_u')
nx.set_edge_attributes(G,  optimism_v, 'optimism_v')

In [31]:
df = pd.DataFrame(columns= ['edge', 'positivity_u', 'positivity_v', "negativity_u", 'negativity_v', 'pesimism_u', 'pesimism_v', 'optimism_u', 'optimism_v', 'sign' ])

In [30]:
positivity_u = nx.get_edge_attributes(G, 'positivity_u')
positivity_v = nx.get_edge_attributes(G, 'positivity_v')
negativity_u = nx.get_edge_attributes(G, 'negativity_u')
negativity_v = nx.get_edge_attributes(G, 'negativity_v')
pesimism_u = nx.get_edge_attributes(G, 'pesimism_u')
pesimism_v = nx.get_edge_attributes(G, 'pesimism_v')
optimism_u = nx.get_edge_attributes(G, 'optimism_u')
optimism_v = nx.get_edge_attributes(G, 'optimism_v')
edge = positivity_u.keys()
sign = nx.get_edge_attributes(G, 'sign')

In [32]:
df['sign'] = sign.values()
df['positivity_u'] = positivity_u.values()
df['positivity_v'] = positivity_v.values()
df['negativity_u'] = negativity_u.values()
df['negativity_v'] = negativity_v.values()
df['pesimism_u'] = pesimism_u.values()
df['pesimism_v'] = pesimism_v.values()
df['optimism_u'] = optimism_u.values()
df['optimism_v'] = optimism_v.values()
df['edge'] = positivity_u.keys()


In [33]:
df.to_csv('emotional_informations.csv')
df

,edge,positivity_u,positivity_v,negativity_u,negativity_v,pesimism_u,pesimism_v,optimism_u,optimism_v,sign
0,"(0, 1)",13,59,4,16,1.0000,0.937500,0.016949,0.016949,1
1,"(0, 2)",13,0,4,0,1.0000,0.000000,0.000000,0.000000,1
2,"(0, 3)",13,9,4,0,1.0000,0.000000,0.000000,0.000000,1
3,"(0, 4)",13,0,4,0,1.0000,0.000000,0.000000,0.000000,1
4,"(0, 5)",13,38,4,4,1.0000,1.000000,0.000000,0.000000,1
5,"(0, 6)",13,10,4,0,1.0000,0.000000,0.000000,0.000000,1
6,"(0, 7)",13,19,4,6,1.0000,0.833333,0.210526,0.210526,1
7,"(0, 8)",13,56,4,58,1.0000,0.931034,0.017857,0.017857,1
8,"(0, 9)",13,17,4,1,1.0000,1.000000,0.000000,0.000000,1
9,"(0, 10)",13,2,4,0,1.0000,0.000000,0.000000,0.000000,1


In [34]:
from sklearn.model_selection import train_test_split
X = df.drop(['edge','sign'],axis=1)
y = df['sign']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
lr = LogisticRegression()
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
lr.score(X_test,y_test)

0.8327218433918118

AttributeError: 'DataFrame' object has no attribute 'isNa'

b###### 